In [1]:
import pandas as pd
import spacy
import json
from tqdm.notebook import tqdm

In [7]:
# Загрузка датасета
df = pd.read_csv('finetuning_text_pairs_3.csv')

In [11]:
# Загрузка модели spaCy для русского языка
nlp = spacy.load('ru_core_news_md')

# Функция для разметки текста и извлечения признаков
def annotate_text(text):
    if pd.isna(text):
        return None
    
    doc = nlp(text)
    
    # Создание структуры для хранения разметки
    annotation = {
        'text': text,
        'tokens': [],
        'sentences': [],
        'entities': [],
        'text_stats': {
            'n_tokens': len(doc),
            'n_sentences': len(list(doc.sents)),
            'avg_token_length': sum(len(token.text) for token in doc) / len(doc) if len(doc) > 0 else 0,
        }
    }
    
    # Добавление информации о токенах
    for token in doc:
        annotation['tokens'].append({
            'text': token.text,
            'lemma': token.lemma_,
            'pos': token.pos_,
            'tag': token.tag_,
            'dep': token.dep_,
            'is_stop': token.is_stop,
            'is_punct': token.is_punct,
            'is_digit': token.is_digit,
        })
    
    # Добавление информации о предложениях
    for sent in doc.sents:
        annotation['sentences'].append({
            'text': sent.text,
            'start_char': sent.start_char,
            'end_char': sent.end_char,
            'n_tokens': len(sent),
        })
    
    # Добавление информации о сущностях
    for ent in doc.ents:
        annotation['entities'].append({
            'text': ent.text,
            'start_char': ent.start_char,
            'end_char': ent.end_char,
            'label': ent.label_,
        })
    
    return annotation

# Выполнение разметки с индикатором прогресса
print("Разметка оригинальных текстов...")
original_annotations = []
for text in tqdm(df['original_text']):
    original_annotations.append(annotate_text(text))

print("Разметка перефразированных текстов...")
paraphrased_annotations = []
for text in tqdm(df['paraphrased_text']):
    paraphrased_annotations.append(annotate_text(text))

# Создание итогового датасета
annotated_data = []
for i in range(len(df)):
    annotated_data.append({
        'original': original_annotations[i],
        'paraphrased': paraphrased_annotations[i]
    })

# Сохранение результатов в JSON
with open('annotated_text_pairs.json', 'w', encoding='utf-8') as f:
    json.dump(annotated_data, f, ensure_ascii=False, indent=2)

print("Разметка завершена. Результаты сохранены в 'annotated_text_pairs.json'")

Разметка оригинальных текстов...


  0%|          | 0/2867 [00:00<?, ?it/s]

Разметка перефразированных текстов...


  0%|          | 0/2867 [00:00<?, ?it/s]

Разметка завершена. Результаты сохранены в 'annotated_text_pairs.json'
